In [1]:
import pandas as pd
df = pd.read_excel(r'C:\Users\a475039\Desktop\TCC Dataset\PR_2019_2023.xlsx')

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from math import sin, cos, pi

In [3]:
df['gravidade'] = df['feridos_leves'] + 3 * df['feridos_graves'] + 5 * df['mortos'] 


In [4]:
df.columns

Index(['Source.Name', 'id', 'data_inversa', 'dia_semana', 'horario', 'uf',
       'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional',
       'delegacia', 'uop', 'gravidade'],
      dtype='object')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'df' is your DataFrame with the relevant columns
# data = df[['gravidade', 'latitude', 'longitude']]
data = df[['latitude', 'longitude']]

# Drop rows with missing values
data = data.dropna()

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Determine the number of clusters using the elbow method (WCSS)
wcss = []
k_values = range(2, 30)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)  # inertia_ gives the WCSS

# Plot the elbow method
plt.figure(figsize=(15, 6))
plt.plot(k_values, wcss, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method')
plt.show()


In [ ]:
# data = df[['gravidade', 'latitude', 'longitude']]
data = df[['latitude', 'longitude']]

# Tratar valores ausentes
data = data.dropna()

# Escalonar os dados
scaler = StandardScaler()
data = scaler.fit_transform(data)


# Determinar o número ideal de clusters
silhouette_scores = []
k_values = range(2, 11)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(data)
    silhouette_scores.append(silhouette_score(data, labels))

# Plotar o método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Coeficiente de Silhouette')
plt.title('Método do Cotovelo')
plt.show()


In [ ]:
optimal_k = 3  # exemplo baseado no gráfico

# Aplicar K-Means
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(data)

# Adicionar os clusters ao dataframe original
df['cluster'] = clusters

# Visualização Geográfica dos Clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='longitude', y='latitude', hue='cluster', data=df, palette='viridis')
plt.title('Clusters de Acidentes')
plt.show()

# Analisar características dos clusters
cluster_summary = df.groupby('cluster').mean()

print(cluster_summary)

In [ ]:
for cluster in range(optimal_k):
    cluster_data = df[df['cluster'] == cluster]
    print(f"Cluster {cluster}:")
    print(cluster_data['gravidade'].describe())

# Análise de Correlação
correlation_matrix = df[['gravidade', 'latitude', 'longitude']].corr()
print("Matriz de Correlação:")
print(correlation_matrix)

# Visualização de Correlações
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação entre Variáveis')
plt.show()

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Análise descritiva e visualização
sns.boxplot(x='tipo_acidente', y='gravidade', data=df)
plt.show()

# # ANOVA
# model = ols('gravidade ~ C(causa_acidente) + C(tipo_acidente) + C(condicao_metereologica) + C(sentido_via) + C(fase_dia)', data=df).fit()
# anova_table = sm.stats.anova_lm(model, typ=2)
# print(anova_table)

# Regressão Linear com variáveis dummy
df = pd.get_dummies(df, drop_first=True)
model = sm.OLS(df['gravidade'], df.drop(columns=['gravidade'])).fit()
print(model.summary())

ValueError: Could not interpret input 'tipo_acidente'

In [11]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

df.columns = df.columns.str.strip()

# Substituir espaços por underscores nos nomes das colunas
df.columns = df.columns.str.replace(' ', '_')

In [12]:
print("Colunas do DataFrame:", df.columns)


Colunas do DataFrame: Index(['id', 'br', 'km', 'pessoas', 'mortos', 'feridos_leves',
       'feridos_graves', 'ilesos', 'ignorados', 'feridos',
       ...
       'uop_UOP04-DEL02-PR', 'uop_UOP04-DEL03-PR', 'uop_UOP04-DEL04-PR',
       'uop_UOP04-DEL05-PR', 'uop_UOP04-DEL07-PR', 'uop_UOP05-DEL01-PR',
       'uop_UOP05-DEL05-PR', 'uop_UOP05-DEL07-PR', 'uop_UOP06-DEL01-PR',
       'uop_UOP07-DEL01-PR'],
      dtype='object', length=3630)


In [ ]:
print(df.isnull().sum())  # Contagem de valores nulos por coluna
# print(np.isinf(df).sum()) 

In [13]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Criação do modelo OLS
model = ols('gravidade ~ C(causa_acidente)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

PatsyError: Error evaluating factor: NameError: name 'causa_acidente' is not defined
    gravidade ~ C(causa_acidente)
                ^^^^^^^^^^^^^^^^^